## 챕터 10: 시퀀스 해킹, 해시, 슬라이스

* 기본 시퀀스 프로토콜: _len_()과 getitem_() 메서드
* 여러 항목을 가진 객체를 안전하게 표현
* 슬라이싱을 지원해서 새로운 벡터 객체 생성
* 포함된 요소 값을 모두 고려한 집합 해싱
* 커스터마이즈된 포맷 언어 확장

### Vector: 사용자 정의 시퀀스형

### Vector 버전 #1: Vector2d 호환
- 최초의 Vector 버전은 앞에서 구현한 Vector2d 클래스와 가능한 한 호환성이 높아야 한다. 그러나 Vector 생성자는 Vector2d 생성자와 호환되지 않도록 설계되어 있다. _init_() 매서드에서 임의의 인수 *args를 받아서 Vector(3, 4)나 Vector(3, 4, 5) 형태로 작동하게 만들 수도 있지만, 시퀀스 생성자는 내장 시퀀스처럼 반복형을 인수로 받게 만드는 것이 좋다.

In [ ]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components) #'보호된' 객체 속성인 self._components는 벡터 요소를 배열로 저장한다.
    
    def __repr__(self):
        components = reprlib.repr(self._components) #반복할 수 있도록 self._components에 대한 반복자를 반환한다.
        components = components[components.find('['):-1] #self._components를 제한된 길이로 표현하기 위해 reprlib.repr()을 사용한다.
        # 즉, array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...]) 형태로 출력한다.
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components)) #self._components에서 바로 bytes 객체를 생성한다.
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self)) #이제는 hypot() 함수를 사용할 수 없으므로, 각 요소의 제곱을 합한 뒤 제곱근을 구하기 위해 sqrt()를 호출한다.
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octects):
        typecode = chr(octects[0])
        memv = memoryview(octects[1:]).cast(typecode)
        return cls(memv)

### 프로토콜과 덕 타이핑
- 객체지향 프로그래밍에서 프로토콜은 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스이다. 예를 들어 파이썬의 시퀀스 프로토콜은 _len_()과 _getitem_() 메서드를 동반할 뿐이다.

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

- FrenchDeck 클래스는 시퀀스 프로토콜을 구현하므로 파이썬에서 제공하는 여러 기능을 활용할 수 있다. 코드 어디에서도 시퀀스 프로토콜을 따른다고 정의한 곳은 없다. 이 클래스가 object를 상속하고 있지만 파이썬 경험이 있는 프로그래머들은 이 코드를 보고 시퀀스라는 것을 알 수 있다.

### Vector 버전 #2: 슬라이스 가능한 시퀀스

#### 슬라이싱의 작동 방식

In [2]:
class MySeq:
    def __getitem__(self, index):
        return index

In [3]:
s = MySeq()
s[1]

1

In [4]:
s[1:4]

slice(1, 4, None)

In [5]:
s[1:4:2]

slice(1, 4, 2)

In [7]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [8]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

**S.indices(len) -> (start, stop, stride)**
: 길이가 len인 시퀀스 S가 나타내는 확장된 슬라이스의 start와 stop 인덱스 및 stride 길이를 계산한다. 경계를 벗어난 인덱스는 일반적인 슬라이스를 처리하는 방법과 동일하게 잘라낸다.
* 즉, indices는 '빠지거나 음수인 인덱스' 그리고 '대상 시퀀스보다 긴 슬라이스'를 우아하게 처리하는 내장된 시퀀스에 구현된 복잡한 논리를 보여준다. 이 메서드는 주어진 길이의 시퀀스 경계안에 들어가도록 조정된 0이나 양수인 start, stop, stride로 구성된 '정규화된' 튜플을 생성한다.

In [9]:
slice(None, 10, 2).indices(5) #'ABCDE'[:10:2]는 'ABCDE[0:5:2]'와 동일하다.

(0, 5, 2)

In [10]:
slice(-3, None, None).indices(5) #'ABCDE'[-3:]은 'ABCDE'[2:5:1]과 동일하다.

(2, 5, 1)

#### 슬라이스를 인식하는 __getitem__()

In [11]:
def __len__(self):
    return len(self._components)

def __getitem__(self, index):
    cls = type(self) # 나중에 사용하기 위해 객체의 클래스(즉, Vector)를 가져온다.
    if isinstance(index, slice): #index 인수가 슬라이스이면,
        return cls(self._components[index]) # _components 배열의 슬라이스로부터 Vector클래스 생성자를 이용해서 Vector객체를 생성한다.
    elif isinstance(index, numbers.Integral): #index 인수가 int 등의 정수형이면(numbers 모듈을 사용하고 있으므로, 이 함수를 사용하려면 numbers 모듈을 임포트해야 한다.)
        return self._components[index] #_components에서 해당 항목을 가져와서 반환한다.
    else:
        msg = '{cls.__name__} indices must be integers'
        raise TypeError(mdg.format(cls=cls))#그렇지 않으면 예외 발생

### Vector 버전 #3: 동적 속성 접근
- Vector2d에서 Vector로 진화하면서 v.x, v.y처럼 벡터 요소를 이름으로 접근하는 능력을 상실했다. 이제는 아주 많은 요소를 가진 벡터를 다루고 있다. 그렇지만 앞에 있는 요소 몇 개는 v[0], v[1], v[2] 대신 x,y,z로 접근할 수 있으면 편리할 것이다.

- Vector2d에서는 @property 데커레이터를 이용해서 x와 y에 읽기 전용 접근을 제공했다. but 이 방법 매우 지루
- _getattr_() 특별 메서드를 이용하면 더욱 깔끔하게 구현할 수 있다.

In [12]:
shortcut_names = 'xyzt'

def __getattr__(self, name):
    cls = type(self) # 나중에 사용하기 위해 Vector 클래스를 가져온다.
    if len(name) == 1: #name이 한 글자면 shortcut_names 중 하나일 수 있다.
        pos = cls.shortcut_names.find(name) #한 글자 name의 위치를 찾는다. str.find()는 'yz'의 위치도 찾을 수 있으므로, 위에서 name의 길이가 1인지 확인한 것이다.
        if 0 <= pos < len(self._components): #position이 범위 안에 있으면 배열 항목을 반환한다.
            return self._components[pos]
    msg = '{.__name__!r} object has no attribute {!r}' #두 개의 검사 과정에 실패하면 표준 메시지와 함께 AttributeError가 발생한다.
    raise AttributeError(msg.format(cls, name))

- 우리가 구현한 _getattr_()은 shortcut_names에 나열된 '가상 속성'의 값을 가져오기 위해 self._components 이외의 다른 속성에는 주의를 기울이지 않는다.

In [13]:
def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1: # 단일 문자 속성명에 대해 특별한 처리를 한다.
        if name in cls.shortcut_names: #name이 x,y,z,t 중 하나면 구체적인 에러 메세지 설정
            error = 'readonly attribute {attr_name!r}'
        elif name.islower(): #name이 그 외 소문자면 단일 문자 속성명에 대한 일반적인 메시지 설정
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else: #그렇지 않으면 error를 빈 문자열로 설정
            error = ''
        if error: #error 안에 어떠한 문자가 들어 있으면 AttributeError를 발생시킨다.
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
    super().__setattr__(name, value) #에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 메서드를 호출한다.

### Vector 버전 #4: 해싱 및 더 빠른==
- _hash_() 메서드를 구현하자. 기존 _eq_() 메서드와 함께 _hash_() 메서드를 구현하면 Vector 객체를 해시할 수 있게 된다.
- _hash_()는 hash(self.x)^(self.y)를 이용해서 해시값을 계산했다. 이제는 hash(v[0])^hash(v[1])^hash(v[2])... 형태로 각 요소의 해시를 계산해서 연속해서 ^(XOR) 연산자를 적용하려고 한다. 바로 이런 곳에 사용하기 위해 functools.reduce()함수가 있다.
- reduce()가 받는 첫 번째 인수는 두 개의 인수를 받는 함수, 두 번째 인수는 반복형이다. 인수 두 개를 받는 함수 fn과 리스트 lst가 있다고 가정하자. reduce(fn, lst)를 호출하면 첫 번째 요소 쌍에 fn을 적용해서(즉, fn(lst[0],lst[1])) 첫 번째 결과 r1을 생성한다. 그러고 나서 r1과 다음 요소에 fn을 적용해서 (즉, fn(r1, lst[2])) 두 번째 결과 r2를 생성한다. 이제 fn(r2, lst[3]))을 호출해서 r3을 생성하고... 이 과정을 마지막 요소까지 반복하면 결국 rN이 반환된다.

In [14]:
2*3*4*5

120

In [15]:
import functools
functools.reduce(lambda a,b: a*b, range(1, 6))

120

In [16]:
# 0에서 5까지 정수를 XOR로 누적 계산하는 세 가지 방법
n = 0
for i in range(6):
    n ^= i

In [17]:
n

1

In [20]:
import functools
functools.reduce(lambda a,b: a^b, range(6))

1

In [21]:
import operator
functools.reduce(operator.xor, range(6))

1

- 맵 단계에서는 각 요소에 대한 해시를 계산하고, 리듀스 단계에서는 모든 해시에 xor 연산자를 적용한다. **제너레이터 표현식** 대신 **맵**을 사용하면 맵 단계가 훨씬 잘 드러난다.

In [22]:
def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes)

In [23]:
# 더 효율적으로 비교하기 위해 for 루프 안에 zip()을 사용한 Vector.__eq__()
def __eq__(self, other):
    if len(self) != len(other): # 두 객체의 길이가 다르면, 객체가 다르다.
        return size
    for a, b in zip(self, other): # zip() 함수는 반복형 인수의 항목으로 구성된 튜플의 제너레이터를 만든다. 
        if a != b: # 두 요소가 다르다는 것이 발견되자마자 False를 반환하면서 빠져나간다.
            return False
    return True # 그렇지 않으면 객체가 동일한 것이다.

In [24]:
# 내장된 zip() 함수의 사용
zip(range(3), 'ABC')

In [25]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [26]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [27]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]